### This is Owen Monroe's Homework Assignment # 3 for Text Mining IS 567
October 8, 2023

1. Train and evaluate a unigram-based baseline classifier

In [20]:
import pandas as pd
import numpy as np
import nltk
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


In [21]:
# Loading Data and Removing Empties

train_df = pd.read_csv('chatgpt_train.csv')
test_df = pd.read_csv('chatgpt_test.csv')

full_train_df = train_df.dropna()
full_test_df = test_df.dropna()

print(len(full_train_df['review'].to_list()))
print(len(full_test_df['review'].to_list()))

full_train_df


1829
458


,date,title,review,rating
0,5/21/2023 16:42,Much more accessible for blind users than the ...,Up to this point I?€?ve mostly been using Chat...,4
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.",I?€?ve been a user since it?€?s initial roll o...,4
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function",This app would almost be perfect if it wasn?€?...,4
3,5/27/2023 21:57,"4.5 stars, here?€?s why","I recently downloaded the app and overall, it'...",4
4,6/9/2023 7:49,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4
...,...,...,...,...
1829,6/4/2023 10:50,pad os???,Please make a iPad version of this,1
1830,5/19/2023 1:23,Goated app,Best app,5
1831,6/21/2023 20:02,Co,Why it?€?s not available in Ethiopia,1
1832,6/7/2023 10:25,Crazy world views,It agrees with letting children be forced into...,1


In [22]:
# Function to Convert Ratings to Three-Way Ratings

def change_five_to_three(rating):
    if rating in [1, 2]:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    elif rating in [4, 5]:
        return 'positive'
    else:
        return 'unknown'

In [23]:
# New Rating List for Train and Test Data

three_rate_train_list = full_train_df['rating'].apply(change_five_to_three).to_list()
three_rate_test_list = full_test_df['rating'].apply(change_five_to_three).to_list()


full_train_df['three rating'] = three_rate_train_list
full_test_df['three rating'] = three_rate_test_list


/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_55994/896341391.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_train_df['three rating'] = three_rate_train_list


In [24]:
full_train_df

,date,title,review,rating,three rating
0,5/21/2023 16:42,Much more accessible for blind users than the ...,Up to this point I?€?ve mostly been using Chat...,4,positive
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.",I?€?ve been a user since it?€?s initial roll o...,4,positive
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function",This app would almost be perfect if it wasn?€?...,4,positive
3,5/27/2023 21:57,"4.5 stars, here?€?s why","I recently downloaded the app and overall, it'...",4,positive
4,6/9/2023 7:49,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4,positive
...,...,...,...,...,...
1829,6/4/2023 10:50,pad os???,Please make a iPad version of this,1,negative
1830,5/19/2023 1:23,Goated app,Best app,5,positive
1831,6/21/2023 20:02,Co,Why it?€?s not available in Ethiopia,1,negative
1832,6/7/2023 10:25,Crazy world views,It agrees with letting children be forced into...,1,negative


In [25]:
full_test_df

,date,title,review,rating,three rating
0,5/19/2023 6:09,error unsupported country,cant login,2,negative
1,5/19/2023 9:39,Hype junk,More harm than help.,1,negative
2,5/19/2023 4:12,your gpt 4 is fake,Fix it,1,negative
3,5/20/2023 3:01,Please impose IPadOS,We need IPadOS!!!,5,positive
4,5/19/2023 20:49,Amazing,Great,5,positive
...,...,...,...,...,...
453,5/19/2023 0:17,Andrew Justino Wilson 5/19/23,This has to be a beginning to something crazy ...,5,positive
454,5/18/2023 19:13,Superb AI,I?€?ve been using chat and have been a proud p...,5,positive
455,5/18/2023 18:27,Fantastic App with Room for Enhancements,The ChatGPT iOS app is an outstanding product....,5,positive
456,5/18/2023 17:17,"Awesome technology, deplorable tactics",Sam Altman?€?s blatant attempt at regulatory c...,2,negative


In [26]:
train_rev_text = full_train_df["review"]
test_rev_text = full_test_df["review"]

# Setting n-gram range to uni-grams
vectorizer = CountVectorizer(ngram_range = (1, 1))

# Creating training data representation
train_rev_uni_cv = vectorizer.fit_transform(train_rev_text.values.astype('U'))
print(train_rev_uni_cv.shape,"\n") 

# Creating test data representation
test_rev_uni_cv = vectorizer.transform(test_rev_text.values.astype('U'))
print(test_rev_uni_cv.shape,"\n") 

print(train_rev_uni_cv)

(1829, 5551) 

(458, 5551) 

  (0, 5224)	1
  (0, 4985)	9
  (0, 4932)	5
  (0, 3613)	1
  (0, 5282)	1
  (0, 3134)	1
  (0, 493)	2
  (0, 5253)	2
  (0, 776)	2
  (0, 3331)	3
  (0, 3165)	2
  (0, 5454)	1
  (0, 1302)	3
  (0, 2139)	1
  (0, 806)	1
  (0, 5421)	1
  (0, 2666)	6
  (0, 1426)	1
  (0, 4267)	3
  (0, 3883)	2
  (0, 3184)	1
  (0, 2660)	3
  (0, 3692)	2
  (0, 1352)	1
  (0, 4906)	16
  :	:
  (1824, 2949)	1
  (1824, 3593)	1
  (1824, 2644)	1
  (1824, 5301)	1
  (1825, 291)	1
  (1825, 527)	1
  (1826, 2666)	1
  (1826, 2456)	1
  (1826, 413)	1
  (1826, 3247)	1
  (1826, 5433)	1
  (1826, 1695)	1
  (1827, 2666)	1
  (1827, 474)	1
  (1827, 5464)	1
  (1827, 2619)	1
  (1827, 3372)	1
  (1827, 1976)	1
  (1827, 2807)	1
  (1827, 794)	1
  (1827, 195)	1
  (1827, 3699)	1
  (1827, 2328)	1
  (1828, 2666)	1
  (1828, 2135)	1


In [27]:
# Define true labels from train set

x_train_cv = train_rev_uni_cv
y_train_rev_series = full_train_df["three rating"]
x_test_cv = test_rev_uni_cv
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv, y_train_rev_series)

# Predict the labels for the test data

predictions = mnb_model.predict(x_test_cv)
predictions

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'negative',
       'positive', 'neutral', 'neutral', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'negative', 'positive'

In [28]:


true_label_list = y_test_rev_series.to_list()
predict_label_list = list(predictions)

In [29]:
# Getting Evaluation Metrics for NB Unigrams 

true_label_list = y_test_rev_series.to_list()
predict_label_list = list(predictions)

acc_score = accuracy_score(true_label_list, predict_label_list)

prec_score_macro = precision_score(true_label_list, predict_label_list, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list, average='weighted')

rec_score_macro = recall_score(true_label_list, predict_label_list, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list, average='weighted')

f1_score_macro = f1_score(true_label_list, predict_label_list, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list, average='weighted')


print('Evaluation Reports - Unigrams NB Baseline')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score for unigrams = {prec_score_macro}')
print(f'micro precision score for unigrams = {prec_score_micro}')
print(f'weighted precision score for unigrams = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score for unigrams = {rec_score_macro}')
print(f'micro recall score for unigrams = {rec_score_micro}')
print(f'weighted recall score for unigrams = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score for unigrams = {f1_score_macro}')
print(f'micro F1 score for unigrams = {f1_score_micro}')
print(f'weighted F1 score for unigrams = {f1_score_weighted}')


Evaluation Reports - Unigrams NB Baseline

Accuracy Score: 0.7358078602620087

Precision Scores
macro precision score for unigrams = 0.5420844714259355
micro precision score for unigrams = 0.7358078602620087
weighted precision score for unigrams = 0.7128098567692405

Recall Scores
macro recall score for unigrams = 0.5007752841737103
micro recall score for unigrams = 0.7358078602620087
weighted recall score for unigrams = 0.7358078602620087

F1 Scores
macro F1 score for unigrams = 0.5116057233704292
micro F1 score for unigrams = 0.7358078602620087
weighted F1 score for unigrams = 0.7176894078769239


2. Remove Features with Low Variance

In [30]:
from sklearn.feature_selection import VarianceThreshold

# Removing Features with threshold below 0.001

selector = VarianceThreshold(threshold = 0.001)

X_train_features_filtered_var_thr_1 = selector.fit(train_rev_uni_cv).transform(train_rev_uni_cv)
print ("Train feature space before filtering: ", train_rev_uni_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr_1.shape)

X_test_features_filtered_var_thr_1 = selector.transform(test_rev_uni_cv)
print ("Test feature space before filtering: ", test_rev_uni_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr_1.shape)

print(X_test_features_filtered_var_thr_1)

Train feature space before filtering:  (1829, 5551)
Train feature space after filtering:  (1829, 3054)
Test feature space before filtering:  (458, 5551)
Test feature space after filtering:  (458, 3054)
  (0, 389)	1
  (0, 1619)	1
  (1, 1243)	1
  (1, 1267)	1
  (1, 1753)	1
  (1, 2682)	1
  (2, 1067)	1
  (2, 1478)	1
  (3, 1467)	1
  (3, 1789)	1
  (3, 2960)	1
  (4, 1204)	1
  (5, 164)	1
  (5, 1187)	1
  (5, 1473)	1
  (5, 2168)	1
  (5, 2706)	1
  (6, 1063)	1
  (7, 164)	1
  (7, 1853)	1
  (8, 1195)	2
  (8, 1473)	1
  (8, 1890)	1
  (8, 2706)	1
  (10, 443)	1
  :	:
  (455, 2912)	1
  (455, 2991)	1
  (455, 3003)	2
  (455, 3004)	1
  (455, 3028)	5
  (456, 62)	1
  (456, 107)	1
  (456, 128)	1
  (456, 214)	1
  (456, 222)	1
  (456, 223)	1
  (456, 281)	1
  (456, 317)	1
  (456, 443)	1
  (456, 772)	1
  (456, 1091)	1
  (456, 1139)	1
  (456, 1846)	1
  (456, 1872)	2
  (456, 2330)	1
  (456, 2483)	1
  (456, 2687)	1
  (456, 2738)	1
  (456, 2972)	1
  (456, 3003)	1


In [31]:
# Training an NB Model for the Feature Space with Low Variance Features Removed threshold below 0.001

# Define true labels from train set

x_train_cv_low1 = X_train_features_filtered_var_thr_1
y_train_rev_series = full_train_df["three rating"]
x_test_cv_low1 = X_test_features_filtered_var_thr_1
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv_low1, y_train_rev_series)

# Predict the labels for the test data

predictions_low1 = mnb_model.predict(x_test_cv_low1)
predictions_low1

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'positive', 'negative',
       'negative', 'negative', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'negative',
       'positive', 'neutral', 'negative', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'negative', 'positive

In [32]:
# Evaluation Metrics for NB with Low Variance Features Removed, Threshold of 0.001

true_label_list = y_test_rev_series.to_list()
predict_label_list_low1 = list(predictions_low1)

acc_score = accuracy_score(true_label_list, predict_label_list_low1)

prec_score_macro = precision_score(true_label_list, predict_label_list_low1, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list_low1, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list_low1, average='weighted')

rec_score_macro = recall_score(true_label_list, predict_label_list_low1, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list_low1, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list_low1, average='weighted')

f1_score_macro = f1_score(true_label_list, predict_label_list_low1, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list_low1, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list_low1, average='weighted')


print('Evaluation Reports - NB with Low Variance Features Removed, Threshold of 0.001 ')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score = {prec_score_macro}')
print(f'micro precision score = {prec_score_micro}')
print(f'weighted precision score = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score = {rec_score_macro}')
print(f'micro recall score = {rec_score_micro}')
print(f'weighted recall score = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score = {f1_score_macro}')
print(f'micro F1 score = {f1_score_micro}')
print(f'weighted F1 score = {f1_score_weighted}')

Evaluation Reports - NB with Low Variance Features Removed, Threshold of 0.001 

Accuracy Score: 0.7270742358078602

Precision Scores
macro precision score for unigrams = 0.5286054212610907
micro precision score for unigrams = 0.7270742358078602
weighted precision score for unigrams = 0.7060217680304899

Recall Scores
macro recall score for unigrams = 0.4974315878104861
micro recall score for unigrams = 0.7270742358078602
weighted recall score for unigrams = 0.7270742358078602

F1 Scores
macro F1 score for unigrams = 0.5065362940263409
micro F1 score for unigrams = 0.7270742358078602
weighted F1 score for unigrams = 0.7117444835328599


In [33]:
selector_5 = VarianceThreshold(threshold = 0.005)

X_train_features_filtered_var_thr_5 = selector_5.fit(train_rev_uni_cv).transform(train_rev_uni_cv)
print ("Train feature space before filtering: ", train_rev_uni_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr_5.shape)

X_test_features_filtered_var_thr_5 = selector_5.transform(test_rev_uni_cv)
print ("Test feature space before filtering: ", test_rev_uni_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr_5.shape)

print(X_test_features_filtered_var_thr_5)

Train feature space before filtering:  (1829, 5551)
Train feature space after filtering:  (1829, 980)
Test feature space before filtering:  (458, 5551)
Test feature space after filtering:  (458, 980)
  (0, 507)	1
  (1, 387)	1
  (1, 546)	1
  (1, 831)	1
  (2, 320)	1
  (2, 459)	1
  (3, 558)	1
  (3, 932)	1
  (4, 367)	1
  (5, 65)	1
  (5, 361)	1
  (5, 455)	1
  (5, 680)	1
  (5, 846)	1
  (6, 318)	1
  (7, 65)	1
  (7, 577)	1
  (8, 364)	2
  (8, 455)	1
  (8, 589)	1
  (8, 846)	1
  (10, 142)	1
  (10, 572)	1
  (10, 579)	1
  (10, 914)	1
  :	:
  (455, 870)	1
  (455, 891)	1
  (455, 900)	3
  (455, 901)	4
  (455, 908)	1
  (455, 913)	1
  (455, 954)	2
  (455, 955)	1
  (455, 966)	5
  (456, 25)	1
  (456, 40)	1
  (456, 83)	1
  (456, 84)	1
  (456, 104)	1
  (456, 142)	1
  (456, 235)	1
  (456, 325)	1
  (456, 342)	1
  (456, 572)	1
  (456, 584)	2
  (456, 775)	1
  (456, 835)	1
  (456, 856)	1
  (456, 936)	1
  (456, 954)	1


In [34]:
# Training an NB Model for the Feature Space with Low Variance Features Removed threshold below 0.005

# Define true labels from train set

x_train_cv_low5 = X_train_features_filtered_var_thr_5
y_train_rev_series = full_train_df["three rating"]
x_test_cv_low5 = X_test_features_filtered_var_thr_5
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv_low5, y_train_rev_series)

# Predict the labels for the test data

predictions_low5 = mnb_model.predict(x_test_cv_low5)
predictions_low5

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'neutral', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'neutral',
       'negative', 'neutral', 'positive', 'negative', 'positive',
       'negative', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'positive', 'neutral', 'negative', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'negative', 'positive',

In [37]:
# Evaluation Metrics for NB with Low Variance Features Removed, Threshold of 0.005

true_label_list = y_test_rev_series.to_list()
predict_label_list_low5 = list(predictions_low5)

acc_score = accuracy_score(true_label_list, predict_label_list_low5)

prec_score_macro = precision_score(true_label_list, predict_label_list_low5, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list_low5, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list_low5, average='weighted')

rec_score_macro = recall_score(true_label_list, predict_label_list_low5, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list_low5, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list_low5, average='weighted')

f1_score_macro = f1_score(true_label_list, predict_label_list_low5, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list_low5, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list_low5, average='weighted')


print('Evaluation Reports - NB with Low Variance Features Removed, Threshold of 0.005 ')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score = {prec_score_macro}')
print(f'micro precision score = {prec_score_micro}')
print(f'weighted precision score = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score = {rec_score_macro}')
print(f'micro recall score = {rec_score_micro}')
print(f'weighted recall score = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score = {f1_score_macro}')
print(f'micro F1 score = {f1_score_micro}')
print(f'weighted F1 score = {f1_score_weighted}')

Evaluation Reports - NB with Low Variance Features Removed, Threshold of 0.005 

Accuracy Score: 0.7358078602620087

Precision Scores
macro precision score = 0.5362426035502958
micro precision score = 0.7358078602620087
weighted precision score = 0.7181904214361385

Recall Scores
macro recall score = 0.4995527704912724
micro recall score = 0.7358078602620087
weighted recall score = 0.7358078602620087

F1 Scores
macro F1 score = 0.5100713456114153
micro F1 score = 0.7358078602620087
weighted F1 score = 0.7195576238803115


3. Select top k-best features using information gain 

In [38]:
# Selecting K-best features with k = 1000

from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=1000)
X_train_features_filtered_kbest1 = selector.fit_transform(x_train_cv, y_train_rev_series)
print ("Train feature space before filtering: ", x_train_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest1.shape)

X_test_features_filtered_kbest1 = selector.transform(x_test_cv)
print ("Test feature space before filtering: ", x_test_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest1.shape)

Train feature space before filtering:  (1829, 5551)
Train feature space after filtering:  (1829, 1000)
Test feature space before filtering:  (458, 5551)
Test feature space after filtering:  (458, 1000)


In [39]:
# Training an NB Model for the Feature Space with K-best features k = 1000

# Define true labels from train set

x_train_cv_k1 = X_train_features_filtered_kbest1
y_train_rev_series = full_train_df["three rating"]
x_test_cv_k1 = X_test_features_filtered_kbest1
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv_k1, y_train_rev_series)

# Predict the labels for the test data

predictions_k1 = mnb_model.predict(x_test_cv_k1)
predictions_k1

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'positive', 'negative',
       'negative', 'negative', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'positive', 'neutral', 'neutral', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'negative', 'positive'

In [41]:
# Evaluation Metrics for NB for the Feature Space with K-best features k = 1000

true_label_list = y_test_rev_series.to_list()
predict_label_list_k1 = list(predictions_k1)

acc_score = accuracy_score(true_label_list, predict_label_list_k1)

prec_score_macro = precision_score(true_label_list, predict_label_list_k1, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list_k1, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list_k1, average='weighted')

rec_score_macro = recall_score(true_label_list, predict_label_list_k1, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list_k1, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list_k1, average='weighted')

f1_score_macro = f1_score(true_label_list, predict_label_list_k1, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list_k1, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list_k1, average='weighted')


print('Evaluation Reports - NB wfor the Feature Space with K-best features k = 1000')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score = {prec_score_macro}')
print(f'micro precision score = {prec_score_micro}')
print(f'weighted precision score = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score = {rec_score_macro}')
print(f'micro recall score = {rec_score_micro}')
print(f'weighted recall score = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score = {f1_score_macro}')
print(f'micro F1 score = {f1_score_micro}')
print(f'weighted F1 score = {f1_score_weighted}')

Evaluation Reports - NB wfor the Feature Space with K-best features k = 1000

Accuracy Score: 0.7489082969432315

Precision Scores
macro precision score = 0.5667439727843752
micro precision score = 0.7489082969432315
weighted precision score = 0.7331256698532228

Recall Scores
macro recall score = 0.5185938663816834
micro recall score = 0.7489082969432315
weighted recall score = 0.7489082969432315

F1 Scores
macro F1 score = 0.5327846831403961
micro F1 score = 0.7489082969432315
weighted F1 score = 0.7320941141075613


In [42]:
# Selecting K-best features with k = 2000

from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=2000)
X_train_features_filtered_kbest2 = selector.fit_transform(x_train_cv, y_train_rev_series)
print ("Train feature space before filtering: ", x_train_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest2.shape)

X_test_features_filtered_kbest2 = selector.transform(x_test_cv)
print ("Test feature space before filtering: ", x_test_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest2.shape)

Train feature space before filtering:  (1829, 5551)
Train feature space after filtering:  (1829, 2000)
Test feature space before filtering:  (458, 5551)
Test feature space after filtering:  (458, 2000)


In [43]:
# Training an NB Model for the Feature Space with K-best features k = 2000

# Define true labels from train set

x_train_cv_k2 = X_train_features_filtered_kbest2
y_train_rev_series = full_train_df["three rating"]
x_test_cv_k2 = X_test_features_filtered_kbest2
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv_k2, y_train_rev_series)

# Predict the labels for the test data

predictions_k2 = mnb_model.predict(x_test_cv_k2)
predictions_k2

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'negative', 'negative', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'positive', 'neutral', 'neutral', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'negative', 'positive'

In [45]:
# Evaluation Metrics for NB for the Feature Space with K-best features k = 2000

true_label_list = y_test_rev_series.to_list()
predict_label_list_k2 = list(predictions_k2)

acc_score = accuracy_score(true_label_list, predict_label_list_k2)

prec_score_macro = precision_score(true_label_list, predict_label_list_k2, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list_k2, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list_k2, average='weighted')

rec_score_macro = recall_score(true_label_list, predict_label_list_k2, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list_k2, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list_k2, average='weighted')

f1_score_macro = f1_score(true_label_list, predict_label_list_k2, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list_k2, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list_k2, average='weighted')


print('Evaluation Reports - NB for the Feature Space with K-best features k = 2000')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score = {prec_score_macro}')
print(f'micro precision score = {prec_score_micro}')
print(f'weighted precision score = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score = {rec_score_macro}')
print(f'micro recall score = {rec_score_micro}')
print(f'weighted recall score = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score = {f1_score_macro}')
print(f'micro F1 score = {f1_score_micro}')
print(f'weighted F1 score = {f1_score_weighted}')

Evaluation Reports - NB for the Feature Space with K-best features k = 2000

Accuracy Score: 0.7379912663755459

Precision Scores
macro precision score = 0.5207628839081421
micro precision score = 0.7379912663755459
weighted precision score = 0.7152427888953939

Recall Scores
macro recall score = 0.48728002849833213
micro recall score = 0.7379912663755459
weighted recall score = 0.7379912663755459

F1 Scores
macro F1 score = 0.49498356240232805
micro F1 score = 0.7379912663755459
weighted F1 score = 0.7179665704150677


4. Lexicon-based feature selection